In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
print(data.head())
data=np.array(data)
np.random.shuffle(data)
train_set=data[:33600,]
print(train_set.shape)
test_set=data[33600:,]
print(test_set.shape)


   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         

In [ ]:
def one_hot(Y):                                   #one hot coding for labels
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

x_train=train_set[:,1:].T
print(x_train.shape)
x_train = x_train / 255.0

y_train=train_set[:,:1].T
y_coded=one_hot(y_train)
print(y_coded[:,60])
print(y_train[:,60])
m=33600

(784, 33600)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[5]


In [ ]:
y_coded.shape

(10, 33600)

In [ ]:
def init_parameters():
    w1=np.random.randn(10, 784) * np.sqrt(2 / 784)
    b1=np.random.rand(10,1)-0.5
    w2=np.random.randn(10, 10) * np.sqrt(2 / 10)
    b2=np.random.rand(10,1)-0.5
    w3=np.random.randn(10, 10) * np.sqrt(2 /10)
    b3=np.random.rand(10,1)-0.5
    return w1,b1,w2,b2,w3,b3

def reLU(z):
    return np.maximum(z,0)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # prevent overflow
    return expZ / np.sum(expZ, axis=0, keepdims=True)



def get_prediction(y_hat):
    return np.argmax(y_hat,axis=0)

def forward_prop(w1,b1,w2,b2,w3,b3,X):
    z1=w1.dot(X)+b1
    a1=reLU(z1)
    z2=w2.dot(a1)+b2
    a2=reLU(z2)
    z3=w3.dot(a2)+b3
    y_hat=softmax(z3)
    return z1,a1,z2,a2,z3,y_hat

def reLU_diff(z):
    return z>0

def back_prop(w1,b1,w2,b2,w3,b3,z1,z2,z3,a1,a2,y_hat,X,Y):
    m=X.shape[1]
    dz3=y_hat-Y
    dw3=(1/m)*(dz3.dot(a2.T))
    db3=(1/m)*(np.sum(dz3,axis=1).reshape(-1,1))

    dz2=(w3.T).dot(dz3)*(reLU_diff(z2))
    dw2=(1/m)*(dz2.dot(a1.T))
    db2=(1/m)*(np.sum(dz2,axis=1).reshape(-1,1))

    dz1=(w2.T).dot(dz2)*(reLU_diff(z1))
    dw1=(1/m)*(dz1.dot(X.T))
    db1=(1/m)*(np.sum(dz1,axis=1).reshape(-1,1))

    return dw3,db3,dw2,db2,dw1,db1

def update(w1,b1,w2,b2,w3,b3,dw3,db3,dw2,db2,dw1,db1,alpha):
    w1=w1-alpha*dw1
    b1=b1-alpha*db1

    w2=w2-alpha*dw2
    b2=b2-alpha*db2

    w3=w3-alpha*dw3
    b3=b3-alpha*db3

    return w1,b1,w2,b2,w3,b3

def get_accuracy(pred,y):
    return np.sum(pred==y)/y.size

def gradient_descent(x,y,alpha,epochs):
    y_coded = one_hot(y.flatten())

    w1,b1,w2,b2,w3,b3=init_parameters()
    for i in range(epochs):
        z1,a1,z2,a2,z3,y_hat=forward_prop(w1,b1,w2,b2,w3,b3,x)
        dw3,db3,dw2,db2,dw1,db1=back_prop(w1,b1,w2,b2,w3,b3,z1,z2,z3,a1,a2,y_hat,x,y_coded)
        w1,b1,w2,b2,w3,b3=update(w1,b1,w2,b2,w3,b3,dw3,db3,dw2,db2,dw1,db1,alpha)
        if(i%20==0):
            print("Iteration no : ",i)
            pred=get_prediction(y_hat)
            accuracy=get_accuracy(pred,y.flatten())*100
            print("accuracy : ",accuracy)

    return w1,b1,w2,b2,w3,b3




In [ ]:
w1,b1,w2,b2,w3,b3=gradient_descent(x_train,y_train,0.05,2000)

Iteration no :  0
accuracy :  10.613095238095237
Iteration no :  20
accuracy :  20.241071428571427
Iteration no :  40
accuracy :  34.23809523809524
Iteration no :  60
accuracy :  60.20833333333333
Iteration no :  80
accuracy :  68.6577380952381
Iteration no :  100
accuracy :  73.09523809523809
Iteration no :  120
accuracy :  76.08035714285715
Iteration no :  140
accuracy :  78.33333333333333
Iteration no :  160
accuracy :  79.92857142857143
Iteration no :  180
accuracy :  81.08928571428572
Iteration no :  200
accuracy :  82.10714285714286
Iteration no :  220
accuracy :  83.14285714285714
Iteration no :  240
accuracy :  83.90773809523809
Iteration no :  260
accuracy :  84.46428571428571
Iteration no :  280
accuracy :  85.05059523809524
Iteration no :  300
accuracy :  85.59821428571428
Iteration no :  320
accuracy :  85.98809523809524
Iteration no :  340
accuracy :  86.42857142857143
Iteration no :  360
accuracy :  86.81547619047619
Iteration no :  380
accuracy :  87.13392857142857
Itera

In [ ]:
x_test = test_set[:,1:].T / 255.0
y_test = test_set[:, :1].T
z1,a1,z2,a2,z3,y_hat = forward_prop(w1,b1,w2,b2,w3,b3,x_test)
pred_test = get_prediction(y_hat)
acc_test = get_accuracy(pred_test, y_test)
print("Test accuracy:", acc_test * 100)


Test accuracy: 92.46428571428571


In [ ]:
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
x_test = np.array(test_data).T / 255.0  # Transpose and normalize
*_, y_hat_test = forward_prop(w1, b1, w2, b2, w3, b3, x_test)
predictions = np.array(get_prediction(y_hat_test))


submission = pd.DataFrame({
    "ImageId": np.arange(1, len(predictions) + 1),
    "Label": predictions
})


submission.to_csv("submission.csv", index=False)